In [9]:
from pathlib import Path
import shutil
from tqdm import tqdm
import sys

#from kilosort.io import load_probe
from kilosort import run_kilosort, DEFAULT_SETTINGS
from kilosort.io import load_probe, save_preprocessing, load_ops

In [ ]:
# quick probe maker
'''
import numpy as np

n_chan = 24
chanMap = np.arange(n_chan)
kcoords = np.zeros(n_chan)
xc = np.ones(n_chan)*7.5
yc = np.array([7.5 + 150*i for i in range(n_chan)])

probe = {
    'chanMap': chanMap,
    'xc': xc,
    'yc': yc[::-1],
    'kcoords': kcoords,
    'n_chan': n_chan
}

print(probe)
'''
#sys.path.append(str(Path('/ihome/pmayo/knoneman/Packages/SGLXMetaToCoords/SGLXMetaToCoords.py').parent))
#from SGLXMetaToCoords import MetaToCoords

# Define the full path to the metadata file
#meta_file_path = Path(f'/ix1/pmayo/lab_NHPdata/{SESSION}_g0/{SESSION}_g0_imec{IMEC_NUM}/{SESSION}_g0_t0.imec{IMEC_NUM}.ap.meta')  # Update with actual file path
#MetaToCoords(metaFullPath=meta_file_path, outType=1, showPlot=True)
#PROBE_PATH = '/ix1/pmayo/lab_NHPdata/kendra_scrappy_0124a_g0/kendra_scrappy_0124a_g0_t0.imec0.ap_kilosortChanMap.mat'

In [10]:
SESSION = 'kendra_scrappy_0127a'
IMEC_NUM = 1

SAVE_PATH = Path(f'/ix1/pmayo/lab_NHPdata/{SESSION}_g0/{SESSION}_g0_imec{IMEC_NUM}/{SESSION}_g0_t0.imec{IMEC_NUM}.ap.bin')
PROBE_PATH = Path(f'/ix1/pmayo/lab_NHPdata/{SESSION}_g0/{SESSION}_g0_imec{IMEC_NUM}/{SESSION}_g0_t0.imec{IMEC_NUM}.ap_kilosortChanMap.mat')

probe = load_probe(PROBE_PATH)

settings = DEFAULT_SETTINGS
print(settings)
settings['probe'] = probe
settings['n_chan_bin'] = probe['n_chan']+1
settings['fs'] = 30000
settings['data_dir'] = SAVE_PATH.parent

# spike detection
#settings['batch_size'] = 30000 # 60000
#settings['nblocks'] = 1 # 0
#settings['Th_universal'] = 9 # 9
#settings['Th_learned'] = 8   # 8

# clustering
#settings['min_template_size'] = 10 #10
#settings['nearest_templates'] = 100 # 100
#settings['max_channel_distance'] = 16 # None

kilosort_path = SAVE_PATH.parent / 'kilosort4'
temp_wh_path = SAVE_PATH.parent / 'temp_wh.dat'

if kilosort_path.exists() and kilosort_path.is_dir():
    shutil.rmtree(kilosort_path)

if temp_wh_path.exists() and temp_wh_path.is_file():
    temp_wh_path.unlink()    

{'n_chan_bin': 385, 'fs': 30000, 'batch_size': 60000, 'nblocks': 1, 'Th_universal': 9, 'Th_learned': 8, 'tmin': 0, 'tmax': inf, 'nt': 61, 'shift': None, 'scale': None, 'artifact_threshold': inf, 'nskip': 25, 'whitening_range': 32, 'highpass_cutoff': 300, 'binning_depth': 5, 'sig_interp': 20, 'drift_smoothing': [0.5, 0.5, 0.5], 'nt0min': None, 'dmin': None, 'dminx': 32, 'min_template_size': 10, 'template_sizes': 5, 'nearest_chans': 10, 'nearest_templates': 100, 'max_channel_distance': 32, 'max_peels': 100, 'templates_from_data': True, 'n_templates': 6, 'n_pcs': 6, 'Th_single_ch': 6, 'acg_threshold': 0.2, 'ccg_threshold': 0.25, 'cluster_downsampling': 20, 'x_centers': None, 'duplicate_spike_ms': 0.25, 'position_limit': 100, 'probe': {'xc': array([ 11., 114.,  11., 114.,  11., 114.,  11., 114.,  11., 114.,  11.,
       114.,  11., 114.,  11., 114.,  11., 114.,  11., 114.,  11., 114.,
        11., 114.,  11., 114.,  11., 114.,  11., 114.,  11., 114.,  11.,
       114.,  11., 114.,  11., 11

In [ ]:
ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = \
    run_kilosort(settings=settings, probe=probe, data_dtype='int16', save_preprocessed_copy=True)

ops_path = SAVE_PATH.parent / 'kilosort4' / 'ops.npy'
ops = load_ops(ops_path)

kilosort.run_kilosort: Kilosort version 0.1.dev1164+gdfaaf6b.d20240502
kilosort.run_kilosort: Python version 3.10.14
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: System information:
kilosort.run_kilosort: Linux-3.10.0-1160.71.1.el7.x86_64-x86_64-with-glibc2.17 x86_64
kilosort.run_kilosort: x86_64
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using CUDA device: NVIDIA A100-PCIE-40GB 39.38GB
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Sorting /ix1/pmayo/lab_NHPdata/kendra_scrappy_0127a_g0/kendra_scrappy_0127a_g0_imec1/kendra_scrappy_0127a_g0_t0.imec1.ap.bin
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 199942711
kilosort.run_kilosort: N seconds: 6664.757033333333
kilosort.run_kilosort: N batches: 3333
ki

In [ ]:
# NOTE: This will only create the .dat file, it will *NOT* update options for Phy.
#       If you want to use this with Phy, you will need to modify `params.py`
#       in the results directory to point to this new file. Additionally,
#       you must set `hp_filtered=True` and `dtype='int16'`.

#save_preprocessing(SAVE_PATH.parent / 'temp_wh.dat', ops, bfile_path=SAVE_PATH)

kilosort.io :  
kilosort.io : ========================================
kilosort.io : Saving drift-corrected copy of data to: /ix1/pmayo/lab_NHPdata/kendra_scrappy_0127a_g0/kendra_scrappy_0127a_g0_imec0/temp_wh.dat...
kilosort.io : Writing batch 0/3333...


## imec1

In [ ]:
SESSION = 'kendra_scrappy_0127a'
IMEC_NUM = 1

SAVE_PATH = Path(f'/ix1/pmayo/lab_NHPdata/{SESSION}_g0/{SESSION}_g0_imec{IMEC_NUM}/{SESSION}_g0_t0.imec{IMEC_NUM}.ap.bin')
PROBE_PATH = Path(f'/ix1/pmayo/lab_NHPdata/{SESSION}_g0/{SESSION}_g0_imec{IMEC_NUM}/{SESSION}_g0_t0.imec{IMEC_NUM}.ap_kilosortChanMap.mat')

probe = load_probe(PROBE_PATH)
print(probe['n_chan'])

settings = DEFAULT_SETTINGS
print(settings)
settings['probe'] = probe
settings['n_chan_bin'] = probe['n_chan']+1
settings['fs'] = 30000
settings['data_dir'] = SAVE_PATH.parent

kilosort_path = SAVE_PATH.parent / 'kilosort4'
temp_wh_path = SAVE_PATH.parent / 'temp_wh.dat'

if kilosort_path.exists() and kilosort_path.is_dir():
    shutil.rmtree(kilosort_path)

if temp_wh_path.exists() and temp_wh_path.is_file():
    temp_wh_path.unlink() 

###################
ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = \
    run_kilosort(settings=settings, probe=probe, data_dtype='int16', save_preprocessed_copy=True)

ops_path = SAVE_PATH.parent / 'kilosort4' / 'ops.npy'
ops = load_ops(ops_path)